# Netcdf to Geotif
- Main goal is to extract and process Copernicus land surface temperature data
- Data Source : https://land.copernicus.eu/global/products/lst
- We have a bunch of files we want to transform from Netcdf to geotif
- later we are going to do some operations as a raster calculator
- Notes: 
   - the netcdf files are in diferent folders
   - the netcdf has multiple variables with multiple dimensions
   - our netcdf has only one band


In [14]:
import netCDF4 as nc
from osgeo import gdal
import numpy as np
import os

In [2]:
"""Make a folder list"""
# path = r"//akif.internal/public/demo-geo-stack/geoserver/data/im-data-soil/isric-bdticm-m-250m-ll_mosaic_10240"
main_path = r"C:\Users\admin\Documents\03_data\Reeni_C0153950"
os.chdir(main_path)
Folder_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
for file in os.listdir(main_path):
    if os.path.isdir(file):
        if file not in Folder_list:
            Folder_list.append(os.path.join(main_path, file).replace("\\","/"))
    else:
        pass

print(Folder_list[0:2])

['C:/Users/admin/Documents/03_data/Reeni_C0153950/LST10_TCI_202101110000_GLOBE_GEO_V2.1.1', 'C:/Users/admin/Documents/03_data/Reeni_C0153950/LST10_TCI_202101210000_GLOBE_GEO_V2.1.1']


In [44]:
"""Main automatized process. Read the rest for further info"""

gdal.UseExceptions()

try: fn.close()  # just to be safe, make sure dataset is not already open.
except: pass
for folder in Folder_list[:]: #1 for testing
    for file in os.listdir(folder):
        if file.endswith(".nc"):
            fn = os.path.join(folder, file).replace("/","\\")#.replace("\\","/")
        else:
            pass

    # Open the nectcdf file, there is only one per folder in this case
    
    ds = nc.Dataset(fn) # If the file is open for write access (mode='w', 'r+' or 'a')
    file_name = os.path.basename(fn)
    output = os.path.join(main_path, file_name).replace(".nc",".tif")
    band = 1
    gdaltranString = "gdal_translate -of GTiff -a_srs epsg:4326 -b " + str(band) + " -co COMPRESS=DEFLATE -co TILED=YES NETCDF:" + str(fn) + ":MEDIAN " + str(output)
    os.system(gdaltranString)
    output = None



In [ ]:
ds = nc.Dataset(fn) # If the file is open for write access (mode='w', 'r+' or 'a')
# Here we print the characteristics
print(ds)
# print(ds.__dict__) # here we print the characteristics as a dictionary

In [35]:
print(os.getcwd())

C:\Users\admin\Documents\03_data\Reeni_C0153950


In [10]:
"""Dimensions check"""
#each variable has its own dimensions, we want MEDIAN
for dim in ds.dimensions.values():
    print(dim) #it is no possible to iterate the dimension

"""Variables check"""
print (ds.variables.keys())

<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 3584

<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 8064

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 1

<class 'netCDF4._netCDF4.Dimension'>: name = 'phony_dim_3', size = 0

odict_keys(['FRAC_VALID_OBS', 'MAX', 'MEDIAN', 'MIN', 'TCI', 'crs', 'lat', 'lon', 'time'])


c:\Users\admin\.conda\envs\gdal_env\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  after removing the cwd from sys.path.


In [33]:
median_variable = ds.variables['MEDIAN']
print(median_variable.dimensions) 

('time', 'lat', 'lon')


In [43]:
"""Here we start transforming the data"""
gdal.UseExceptions()

file_name = os.path.basename(fn)
output = os.path.join(main_path, file_name).replace(".nc",".tif")
band = 1

# nn = gdal.Translate(output, fn, 
#                 format='GTIFF',
#                 # format='NetCDF',
#                 # bandList = [1]
#                 # creationOptions = ['COMPRESS=DEFLATE','TILED=YES']
#                 )

gdaltranString = "gdal_translate -of GTiff -a_srs epsg:4326 -b " + str(band) + " -co COMPRESS=DEFLATE -co TILED=YES NETCDF:" + str(fn) + ":MEDIAN " + str(output)
# gdaltranString = "gdal_translate -of GTiff -a_srs epsg:4326 -b " + str(1) + " -co COMPRESS=DEFLATE -co TILED=YES " + str(fn) + " " + str(output)

os.system(gdaltranString)
output = None

#gdal_translate -of GTiff -b 10 input.nc output.tiff  # to get 10th band, it works fine

https://cds.climate.copernicus.eu/cdsapp#!/dataset/derived-utci-historical?tab=overview